# Storing Temporal Data
In general, the value of **time series** data is in its retrospective (batch ingestion model), rather than in the live streaming of data. For this reason, storing **time series** data is necessary for most analyses.

A good storage solution is one that allows for easy access and reliability of data without requiring a large investment of computing resources. Later, we will look at what aspects of a dataset we should consider for storage, as well as examine the advantages of SQL databases, NoSQL databases, and a variety of flat file formats.